Hate Speech Detection uygulamız için modelimizi kaydedeceğiz

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
# Veriyi Yükleyip EDA yapıyoruz

In [4]:
df = pd.read_csv("twitter.csv")

In [5]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [7]:
df.isnull().sum()

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [8]:
# Veri Temizleme Fonksiyonu

In [9]:
def clean_text(text):
    text = str(text).lower() # Küçük harfe çevir
    text = re.sub('\[.*?\]', '', text) # Köşeli parantez içindekileri sil
    text = re.sub('https?://\S+|www\.\S+', '', text) # URL'leri sil
    text = re.sub('<.*?>+', '', text) # HTML etiketlerini sil
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Noktalama işaretlerini sil
    text = re.sub('\n', '', text) # Yeni satır karakterlerini sil
    text = re.sub('\w*\d\w*', '', text) # Rakam içeren kelimeleri sil
    return text

In [10]:
# Temizleme işlemini uyguluyoruz

In [11]:
import re

In [12]:
import string

In [13]:
df['clean_tweet'] = df['tweet'].apply(clean_text)

In [14]:
# Model Hazırlığı

In [15]:
X = df['clean_tweet']

In [16]:
y = df['class']

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Pipeline Kurulumu (Vektörleştirme + Model)

In [20]:
from sklearn.pipeline import Pipeline

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
from sklearn.tree import DecisionTreeClassifier

In [23]:
# CountVectorizer metni sayıya çevirir, DecisionTree sınıflandırır.

In [24]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')), # İngilizce etkisiz kelimeleri (the, a, in) çıkar
    ('classifier', DecisionTreeClassifier(random_state=42))
])

In [25]:
# Modeli Eğitme

In [26]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(stop_words='english')),
                ('classifier', DecisionTreeClassifier(random_state=42))])

In [27]:
pipeline.score(X_train, y_train)

0.9992938565520024

In [28]:
# Değerlendirme

In [29]:
y_pred = pipeline.predict(X_test)

In [30]:
from sklearn.metrics import accuracy_score, classification_report

In [31]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Accuracy: 0.8735


In [32]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.37      0.32      0.34       290
           1       0.93      0.92      0.92      3832
           2       0.79      0.84      0.81       835

    accuracy                           0.87      4957
   macro avg       0.70      0.69      0.69      4957
weighted avg       0.87      0.87      0.87      4957



In [33]:
# Model Save

In [34]:
import joblib

In [35]:
joblib.dump(pipeline, 'hate_speech_model.pkl')

['hate_speech_model.pkl']

pipeline.score: 0.99        
Accuracy: 0.8735